In [ ]:
#!/usr/bin/env python3
"""mock_continuous_signal.ipynb
James Gardner 2019
Melbourne Uni
"""

# import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})
# %matplotlib inline
from tqdm import tqdm_notebook as tqdm

In [ ]:
class SineSignal(object):
    n_t = int(1e3)
    # ultimately, freq_prob will be half this size
    t = np.linspace(0,2*np.pi,n_t)    
    dt = t[1]-t[0]
    noise_scale = 7
    
    def __init__(self,raw_freq):
        self.raw_freq = raw_freq
        self.raw_signal = np.sin(raw_freq*2*np.pi*SineSignal.t)
        self.noise = np.random.normal(0,SineSignal.noise_scale,self.raw_signal.shape)
        self.injected_signal = self.raw_signal + self.noise
        self.yf = np.fft.fft(self.injected_signal)
        self.nrm_abs_yf = 2/SineSignal.n_t*np.abs(self.yf)
        self.inj_freq = np.fft.fftfreq(len(self.yf),SineSignal.dt)
        # np.fft.fftfreq outputs 0 to +inf then -inf to 0, so :N//2 gets +ve side; wild!
        self.freq_prob = self.nrm_abs_yf[:SineSignal.n_t//2]
        
    def plot(self):
        plt.figure(figsize=(14,7))
        plt.plot(SineSignal.t,self.injected_signal)
        plt.title("injected signal")
        plt.ylabel("signal value")
        plt.xlabel("signal time chunk")
        plt.show()
        plt.figure(figsize=(14,7))        
        plt.plot(self.inj_freq[:SineSignal.n_t//2],self.freq_prob)
        plt.title("discrete FFT")
        plt.ylabel("freq strength in signal")                
        plt.xlabel("frequency")
        plt.show()

In [ ]:
# SineSignal(5).plot()

In [ ]:
long_timesteps = 100
bin_time = np.linspace(0,1,long_timesteps)
# again halve total number of points due to fft on real function
grid = np.zeros((SineSignal.n_t//2,long_timesteps))
print('grid is:',grid.shape)

meander_amp = 20
meander_decay = 2
meander_freq = 2
meander = lambda x: meander_amp*np.exp(-x*meander_decay)*np.sin(meander_freq*2*np.pi*x)

wandering_freqs = meander(bin_time)+20

post_freq = []

for i,f in tqdm(enumerate(wandering_freqs)):
    thing_f = SineSignal(f)
    col = thing_f.freq_prob
    grid[:,i] = col
    post_freq.append(thing_f.inj_freq[col.argmax()])

ngrid = grid/np.max(grid)

lngrid = np.log(ngrid)

In [ ]:
plt.figure(figsize=(14,7))
plt.plot(bin_time,meander(bin_time)+20,'.')    
plt.plot(bin_time,post_freq,'r.')
plt.ylabel("signal frequency")
plt.xlabel("bin time")
plt.savefig('meander.pdf',bbox_inches='tight')
plt.clf()

# plt.figure(figsize=(70,35))
plt.figure(figsize=(7,14))
plt.imshow(ngrid, cmap='viridis')
plt.gca().xaxis.tick_top()
plt.colorbar()
plt.ylabel('signal frequency bins')
plt.xlabel('long time bins')
plt.savefig('wandering.pdf',bbox_inches='tight')
plt.clf()

plt.figure(figsize=(7,14))
plt.imshow(lngrid, cmap='viridis')
plt.gca().xaxis.tick_top()
plt.colorbar()
plt.ylabel('signal frequency bins')
plt.xlabel('long time bins')
plt.savefig('lnwandering.pdf',bbox_inches='tight')
plt.clf()

In [ ]:
# score_tmp = np.zeros(np.shape(lngrid))
# score_tmp[:,0] = lngrid[:,0]
score_tmp  = np.copy(lngrid)
pathfinder = np.full(np.shape(lngrid), 500)
pathfinder[:,0] = 501

In [ ]:
scanning_range = 10

for j in tqdm(range(1,long_timesteps)): #range(100)
    for i in range(len(score_tmp[:,j])): #range(500)
        k_a = max(0, i-scanning_range) 
        k_b = min(len(score_tmp[:,j-1])-1, i+scanning_range)
        #print(k_a,k_b)
        window = score_tmp[:,j-1][k_a:k_b+1]
        window_score = np.max(window)
        window_ref   = k_a+np.argmax(window)

        # [i][j] same as [:,j][i] same as [i,j]
        score_tmp[i][j] += window_score
        pathfinder[i][j] = window_ref 



In [ ]:
# (score_tmp-lngrid)[:,1]

In [ ]:
best_score  = np.max(score_tmp[:,long_timesteps-1])
best_end = np.argmax(score_tmp[:,long_timesteps-1])
best_path_back = np.full(long_timesteps,502)
best_path_back[long_timesteps-1] = best_end

path_grid = np.zeros(np.shape(ngrid))
tmp_path = pathfinder[best_end][long_timesteps-1]

for j in tqdm(reversed(range(0,long_timesteps-1))):
    path_grid[tmp_path][j] = 1
    best_path_back[j] = tmp_path    
    tmp_path = pathfinder[tmp_path][j]

# 501 is the pathfinder flag for reaching the first, 0-index column
assert tmp_path == 501

plt.figure(figsize=(7,14))
plt.imshow(path_grid, cmap='viridis')
plt.gca().xaxis.tick_top()
plt.colorbar()
plt.ylabel('signal frequency bins')
plt.xlabel('long time bins')
plt.savefig('viterbi_path.pdf',bbox_inches='tight')
plt.clf()

In [ ]:
# scanning_range = 10

# for i in tqdm(range(len(score_tmp[:,1]))): #range(500)
#     k_a = max(0, i-scanning_range) 
#     k_b = min(len(score_tmp[:,0])-1, i+scanning_range)
#     #print(k_a,k_b)
#     back_score = np.max(score_tmp[:,0][k_a:k_b+1])
#     from_where = np.argmax(score_tmp[:,0][k_a:k_b+1])
    
#     # [i][1] same as [:,1][i]
#     score_tmp[i][1] += back_score
#     pathfinder[i][1] = from_where 

In [ ]:
# n_t = int(1e3)
# # ultimately, freq_prob will be half this size
# t = np.linspace(0,2*np.pi,n_t)
# dt = t[1]-t[0]
# raw_freq = 20

# raw_signal = np.sin(raw_freq*2*np.pi*t)

# noise_scale = 0.2
# noise = np.random.normal(0,noise_scale,raw_signal.shape)

# injected_signal = raw_signal + noise
# # plt.plot(t,injected_signal)
# # plt.show()

# yf = np.fft.fft(injected_signal)
# nrm_abs_yf = 2/n_t*np.abs(yf)
# inj_freq = np.fft.fftfreq(len(yf),dt)
# # np.fft.fftfreq outputs 0 to +inf then -inf to 0, so :N//2 gets +ve side; wild!
# plt.plot(inj_freq[:n_t//2],nrm_abs_yf[:n_t//2])
# freq_prob = nrm_abs_yf[:n_t//2]

# plt.show()

In [ ]:
# https://stackoverflow.com/questions/25735153/plotting-a-fast-fourier-transform-in-python
# N = 600
# # t = np.arange(N)/N*2*np.pi
# # t = np.arange(0,2*np.pi,T)
# t = np.linspace(0,2*np.pi,N)
# T = t[1]-t[0]

# y = np.sin(50*2*np.pi*t) + 0.5*np.sin(80*2*np.pi*t)
# # plt.plot(t,y)
# # plt.show()

# # nrm_abs_yf = 2/n_t*np.abs(yf)
# # inj_freq = np.fft.fftfreq(len(yf),dt)
# # plt.plot(inj_freq,nrm_abs_yf)

# yf = np.fft.fft(y)
# xf = np.linspace(0, 1/(2*T), N//2)
# plt.plot(xf, 2/N * np.abs(yf[:N//2]))
# plt.show()